In [3]:
from bs4 import BeautifulSoup
from requests import TooManyRedirects
import re
import requests
from datetime import datetime
import os
import pandas as pd
import regex as re

In [4]:
def get_critic_page_init(movie_name):
    movie_name = re.sub(r"[^a-zA-Z]{1,}", "_", movie_name).lower()
    if movie_name[-1] == '_':
        movie_name = movie_name[:-1]
    print(f'Try to get response from https://www.rottentomatoes.com/m/{movie_name}/reviews')
    response = requests.get(f'https://www.rottentomatoes.com/m/{movie_name}/reviews')
    soup = BeautifulSoup(response.content, 'html.parser')
    critics = soup.find_all('div',class_ = 'review-row')

    # find movie id
    rtid = soup.find('script',{"id":"mps-page-integration"}).contents[0].replace('|','').strip()
    rtid = [x for x in rtid.split(',') if 'rtid' in x][0].strip().split(':')[-1][1:-1]

    load_btn = soup.find_all('rt-button',{'data-loadmoremanager':"btnLoadMore:click"})
    if load_btn:
        hasNextPage = True
    else:
        hasNextPage = False
    return critics,rtid, hasNextPage

In [5]:
def get_critic_page_follow(movie_id,start_token = None):
    url = f'https://www.rottentomatoes.com/napi/movie/{movie_id}/reviews/all?after={start_token}%3D%3D&pageCount=20'
    print(f'Using url:{url}')
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print('Aborted as response code is not 200')
            return None
    except error as e:
        print(f'[request {url} failed] : {e}')
    return json.loads(response.content)

In [6]:
def get_review_data_soup(critics):
    critics_reviewer = critics.find_all('div',class_ = 'reviewer-name-and-publication')
    disp_name = critics_reviewer[0].find( class_ = "display-name").contents[0].strip()
    # find review_date
    critics_score_content = [x for x in critics.find('p',class_ = 'original-score-and-url').contents if hasattr(x,'contents')]
    def try_dt_map(x):
        try:
            dt = datetime.strptime(x.contents[0], '%b %d, %Y')
            return dt
        except:
            return None
    review_date = [dt for dt in list(map(try_dt_map,critics_score_content)) if bool(dt)]
    if review_date:
        review_date = review_date[0]
    else:
        review_date = 'Not found'
        
    # find score
    critics_score_str = [x for x in critics.find('p',class_ = 'original-score-and-url').contents if not hasattr(x,'contents')]
    def try_score_map(x):
        try:
            score = x.replace('|','').strip()
            return score
        except:
            return None
    review_score = [score for score in list(map(try_score_map,critics_score_str)) if bool(score)]
    if review_score:
        review_score = review_score[0].split()[-1].strip()
    else:
        review_score = 'Not found'

    # find sentiment
    review_sentiment = critics.find('score-icon-critics').attrs['sentiment']
    # find review text
    review_text = critics.find('p', class_ = 'review-text').contents[0]
    if review_text:
        review_text = review_text
    else:
        review_text = 'Not Found'
        
    return [disp_name,review_date,review_score,review_sentiment,review_text,1]
                      

In [7]:
def get_review_data_json(json_rvw_data, page = None):
    def get_json_info(jsn,key):
        if key in jsn.keys():
            return jsn[key]
        else:
            return None
    disp_name = get_json_info(json_rvw_data,'criticName')
    review_date = get_json_info(json_rvw_data, 'creationDate')
    review_date = datetime.strptime(json_rvw_data['creationDate'], '%b %d, %Y')
    review_score = get_json_info(json_rvw_data, 'originalScore')
    review_sentiment = get_json_info(json_rvw_data, 'scoreSentiment')
    review_text = get_json_info(json_rvw_data, 'quote')

    return [disp_name,review_date,review_score,review_sentiment,review_text,page]

In [9]:
def get_movie_reviews(movie_title):
    # create dataframe
    df = pd.DataFrame({'movie_title':[],
                       'movie_id':[],
                       'reviewer_name':[],
                       'review_date':[],
                       'review_score':[],
                       'review_sentiment':[],
                       'review_text':[],
                       'on_page':[]
                        })
    def append_review_df(df, movie_title, movie_id, reviewer_name, review_date, review_score, review_sentiment, review_text, on_page):
        df_to_append = pd.DataFrame({'movie_title':[movie_title],
                                     'movie_id':[movie_id],
                                     'reviewer_name':[reviewer_name],
                                     'review_date':[review_date],
                                     'review_score':[review_score],
                                     'review_sentiment':[review_sentiment],
                                     'review_text':[review_text],
                                     'on_page':[on_page]
                                    })
        return df._append(df_to_append, ignore_index = True)
    # page1 init
    init_pg_critics, rtid, hasNextPage = get_critic_page_init(movie_title)
    # ingest page1 data
    print(f'Scraping page{1} of {movie_title}')
    for row in range(len(init_pg_critics)):
        review_row = init_pg_critics[row]
        rtid = get_critic_page_init(movie_title)[1]
        review_row_data = get_review_data_soup(review_row)
        df = append_review_df(df, movie_title, rtid, *review_row_data)
    # end if no load more btn found
    if not hasNextPage:
        return df
    
    # other page
    is_first_loop = True
    while True:   
        if is_first_loop:
            start_token = 'MQ'
        res = get_critic_page_follow(movie_id = rtid,start_token = start_token)
        if not res: # the movie has no more review page
            break
        pg = 2 # page count
        print(f'Scraping page{pg} of {movie_title} using start key = {start_token}')
        for row in res['reviews']:
            review_row_data = get_review_data_json(row, page = pg)
            df = append_review_df(df, movie_title, rtid, *review_row_data)
        is_first_loop = False # Disable init var
        # check if has next page
        print(res['pageInfo']['hasNextPage'])
        if not res['pageInfo']['hasNextPage']:
            return df      
        start_token = res['pageInfo']['endCursor'].replace('==','')
        print(f'Found next page token {start_token}')    
    return df

In [11]:
x = get_movie_reviews('Venom: The Last Dance')

x

Try to get response from https://www.rottentomatoes.com/m/venom_the_last_dance/reviews
Scraping page1 of Venom: The Last Dance
Try to get response from https://www.rottentomatoes.com/m/venom_the_last_dance/reviews
Try to get response from https://www.rottentomatoes.com/m/venom_the_last_dance/reviews


C:\Users\User\AppData\Local\Temp\ipykernel_3204\3163890443.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return df._append(df_to_append, ignore_index = True)


Try to get response from https://www.rottentomatoes.com/m/venom_the_last_dance/reviews
Try to get response from https://www.rottentomatoes.com/m/venom_the_last_dance/reviews
Try to get response from https://www.rottentomatoes.com/m/venom_the_last_dance/reviews
Try to get response from https://www.rottentomatoes.com/m/venom_the_last_dance/reviews
Try to get response from https://www.rottentomatoes.com/m/venom_the_last_dance/reviews
Try to get response from https://www.rottentomatoes.com/m/venom_the_last_dance/reviews
Try to get response from https://www.rottentomatoes.com/m/venom_the_last_dance/reviews
Try to get response from https://www.rottentomatoes.com/m/venom_the_last_dance/reviews
Try to get response from https://www.rottentomatoes.com/m/venom_the_last_dance/reviews
Try to get response from https://www.rottentomatoes.com/m/venom_the_last_dance/reviews
Try to get response from https://www.rottentomatoes.com/m/venom_the_last_dance/reviews
Try to get response from https://www.rotten

,movie_title,movie_id,reviewer_name,review_date,review_score,review_sentiment,review_text,on_page
0,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Rick Bentley,2024-11-24,C+,POSITIVE,"As far as road trip tales go, “Venom: The Last...",1.0
1,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Gayle Sequeira,2024-11-23,Not found,POSITIVE,"Playing Eddie and Venom, Tom Hardy wrings a su...",1.0
2,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Mark Kermode,2024-11-22,Not found,NEGATIVE,An absolute car crash of a film. That's not to...,1.0
3,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Cris Kennedy,2024-11-22,3/5,POSITIVE,Marcel does understand that humour is the key ...,1.0
4,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Nick Bhasin,2024-11-20,Not found,NEGATIVE,So it's entirely possible that this movie felt...,1.0
...,...,...,...,...,...,...,...,...
203,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Casey Chong,2024-10-23,1.5/5,NEGATIVE,A futile third Venom movie that fails to give ...,2.0
204,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Jake Cole,2024-10-23,3/4,POSITIVE,As the film progresses&#44; it consistently es...,2.0
205,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Elliott Collins,2024-10-23,2/5,NEGATIVE,In a whirlwind of chaotic creativity&#44; Veno...,2.0
206,Venom: The Last Dance,0f101f8c-ec09-39c4-9be0-2f9cc464d332,Soren Andersen,2024-10-23,1.5/4,NEGATIVE,“The Last Dance” brings nothing new to the ser...,2.0


In [40]:
x = get_movie_reviews('the_ring')
x

Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Scraping page1 of the_ring
Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews


C:\Users\User\AppData\Local\Temp\ipykernel_3204\3163890443.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return df._append(df_to_append, ignore_index = True)


Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Try to get response from https://www.rottentomatoes.com/m/the_ring/reviews
Try to get response from 

,movie_title,movie_id,reviewer_name,review_date,review_score,review_sentiment,review_text,on_page
0,the_ring,75a5d137-c175-34eb-844c-1a0fb1cd9be5,Elisa Guimarães,2024-01-29,Not found,POSITIVE,The Ring is one of those rare movies that gain...,1.0
1,the_ring,75a5d137-c175-34eb-844c-1a0fb1cd9be5,Kristy Strouse,2023-11-08,Not found,POSITIVE,The Ring captivates audiences with its origina...,1.0
2,the_ring,75a5d137-c175-34eb-844c-1a0fb1cd9be5,Rick Romancito,2023-10-29,B+,POSITIVE,"It created a myth, one that drew upon the supe...",1.0
3,the_ring,75a5d137-c175-34eb-844c-1a0fb1cd9be5,Stephanie Archer,2023-07-01,Not found,POSITIVE,The Ring still shines in its own era of horror...,1.0
4,the_ring,75a5d137-c175-34eb-844c-1a0fb1cd9be5,Kent Garrison,2022-09-26,A-,POSITIVE,The creepy video tape holds up as still very c...,1.0
...,...,...,...,...,...,...,...,...
205,the_ring,75a5d137-c175-34eb-844c-1a0fb1cd9be5,Robin Clifford,2002-10-07,C-,NEGATIVE,"The result is a film that lacks the coherence,...",2.0
206,the_ring,75a5d137-c175-34eb-844c-1a0fb1cd9be5,Kirk Honeycutt,2002-10-04,,POSITIVE,"An undeniably creepy, unnerving experience tha...",2.0
207,the_ring,75a5d137-c175-34eb-844c-1a0fb1cd9be5,David Nusair,2002-10-01,3/4,POSITIVE,...The Ring is far scarier than any of the rec...,2.0
208,the_ring,75a5d137-c175-34eb-844c-1a0fb1cd9be5,Mark Ramsey,2002-09-20,3/4,POSITIVE,It's Die-per-View! This videotape wants to be ...,2.0
